<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/adapters_confirm_classification_no_pretraining_20_percent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This is just a check to confirm that we can reload and get the same results.

# Result
**Matches!**

### Previous

'eval_loss': 0.33062857389450073

'eval_accuracy': 0.8662

'eval_f1_macro': 0.6273391985352926

'eval_runtime': 37.7912

### Results

'eval_loss': 0.33062857389450073

'eval_accuracy': 0.8662

'eval_f1_macro': 0.6273391985352926

'eval_runtime': 38.6531



# Setup

In [16]:
# Classification Dataset Settings
classification_dataset_name = "BigTMiami/amazon_helpfulness"
helpfulness_percentage = "5"
train_split = f"train[:{helpfulness_percentage}%]"
eval_split = "dev"

# Adapter to load from hub
adapter_model_hub_name = "BigTMiami/adapter_classification_no_pretraining_20_percent"

# Classification Model Hub Name if pushed \ Output Dir
classification_model_hub_name = "classification_model_hub_name"



In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.0 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [10]:
from datasets import load_dataset

classification_train_dataset = load_dataset(classification_dataset_name, split=train_split)
classification_eval_dataset = load_dataset(classification_dataset_name, split=eval_split)

print(classification_train_dataset)
print(classification_eval_dataset)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5763
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [12]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

# Load from hub
config = RobertaConfig.from_pretrained("roberta-base")
classification_model = AutoAdapterModel.from_pretrained("roberta-base",config=config)

adapter_name = classification_model.load_adapter(adapter_model_hub_name)
classification_model.set_active_adapters([adapter_name])


Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# adapter_hub_name = "adapter_classifier_from_prtrained_test"
# adapter_name = model.load_adapter("BigTMiami/domain_adapter_test")
# # Add head for masked language modeling
# model.add_classification_head(
#     adapter_name,
#     num_labels=2,
#     id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
#   )

# # Set the adapter to be used for training
# model.train_adapter(adapter_name)

In [ ]:
# adapter_hub_name = "adapter_classification_no_pretraining_20_percent"
# adapter_name = "classifier_seq_bn_no_pretraining_20_percent"
# adapter_type = "seq_bn" # could be "lora", etc.

# # Add a new adapter
# model.add_adapter(adapter_name, config=adapter_type)

# # Add head for classification modeling
# model.add_classification_head(
#     adapter_name,
#     num_labels=2,
#     id2label={ 0: "unhelpful", 1: "helpful"})

# # Set the adapter to be used for training
# model.train_adapter(adapter_name)

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=classification_model_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [18]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=classification_model,
    args=training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


GPU 1403 MB


In [ ]:
# training_results = trainer.train()
# print_gpu_utilization()
# print(training_results)

In [19]:
eval_results = trainer.evaluate()

eval_loss = eval_results["eval_loss"]
eval_f1_macro = eval_results["eval_f1_macro"]
print_gpu_utilization()
print(f"Trained eval_f1_macro:{100.0 * eval_f1_macro:.2f} Eval Loss: {eval_loss:.2f}")
print(eval_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


GPU 2427 MB
Trained eval_f1_macro:62.73 Eval Loss: 0.33
{'eval_loss': 0.33062857389450073, 'eval_accuracy': 0.8662, 'eval_f1_macro': 0.6273391985352926, 'eval_runtime': 38.6531, 'eval_samples_per_second': 129.356, 'eval_steps_per_second': 8.098}


In [ ]:
# model.push_adapter_to_hub(
#     adapter_hub_name,
#     adapter_name,
#     datasets_tag=dataset_name
# )

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()